In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [153]:
import fiona
import collections
import matplotlib.pyplot as plt
import pyproj
import rasterio
import rasterio.mask
from rasterio.plot import show
from rasterio import features
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date, timedelta
import geopandas
import os
from shapely import geometry
import numpy as np
import utilities
import pandas as pd
import shutil
import glob
pd.set_option('display.max_columns', None)
import time
import zipfile
import geopandas as gpd
import shapely
import geopandas as gpd
from shapely.geometry import MultiPoint, Polygon

In [180]:
# 18-21 Jul 2019
start_date = date.today() - timedelta(10)
end_date = date.today()

In [155]:
path = "/home/primity/terras/bulk/"
dest = "/home/primity/terras/bulk_crops/"

In [156]:
freguesias_shapes=fiona.open("gadm36_PRT_shp/gadm36_PRT_3.shp")
concelhos_shapes = fiona.open("gadm36_PRT_shp/gadm36_PRT_2.shp")

for shp in freguesias_shapes:
    #print(shp['properties']['NAME_3'])
    if shp['properties']['NAME_3']=='Cardigos' :
        cardigos_shp = shp

for shp in concelhos_shapes:
    if shp['properties']['NAME_2'] =='Mação':
        macao_shp = shp
        
macao_outer_square = utilities.outer_square_points(macao_shp['geometry']['coordinates'][0])

In [189]:
os.makedirs(path, exist_ok=True)
os.makedirs(dest, exist_ok=True)


#get products list from this day
api = SentinelAPI('fernandeslouro', 'copernicospw', 'https://scihub.copernicus.eu/dhus')
products = api.query(macao_outer_square,
                     date=(start_date, end_date),
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 30))
products_df = api.to_dataframe(products)

products_df['footprint'] = geopandas.GeoSeries.from_wkt(products_df['footprint'])
products_df = gpd.GeoDataFrame(products_df, geometry='footprint')
products_df['centroid_distances'] = products_df.apply(lambda row: row.footprint.centroid.distance(macao_outer_square.centroid), axis=1)

print(f"{current_date} - {len(products)} available products")
# download them all
to_download = products_df[products_df.centroid_distances == min(products_df.centroid_distances)].sample(1)
to_download = utilities.single_row_dataframe_to_dict(to_download)

utilities.download_no_fail(to_download['uuid'], path, 'fernandeslouro', 'copernicospw', 'https://scihub.copernicus.eu/dhus')

with zipfile.ZipFile(os.path.join(path, to_download['title']+'.zip'), 'r') as zip_ref:
    zip_ref.extractall(path)

os.remove(os.path.join(path, to_download['title']+'.zip'))

In [199]:
# copy jp2 to dest
utilities.subfolders_copy(os.path.join(path, to_download['title'] + '.SAFE'), dest)

# save cropped jp2 with same name 
for i in os.listdir(dest):
    with rasterio.open(os.path.join(dest, i)) as src:
        out_image, out_transform = rasterio.mask.mask(src, utilities.transform_shapefile(macao_shp), crop=True, nodata=10, all_touched=True)
        out_meta = src.meta.copy() 
        
    with rasterio.open(os.path.join(dest, i), 'w', dtype=rasterio.uint8, count=1,compress='lzw') as dst:
        dst.write(out_image)
        
for f in glob.glob(os.path.join(path, to_download['title'])):
    os.remove(f)
 

TypeError: Integer width and height are required.